<a href="https://colab.research.google.com/github/plaban1981/Haystack_NLP/blob/main/haystack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install urllib3==1.25.4

     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 1.2MB 34.8MB/s 
     |████████████████████████████████| 10.1MB 36.5MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 7.2MB 31.8MB/s 
     |████████████████████████████████| 2.5MB 31.7MB/s 
     |████████████████████████████████| 184kB 40.5MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 1.1MB 30.5MB/s 
     |████████████████████████████████| 153kB 37.8MB/s 
     |████████████████████████████████| 348kB 39.5MB/s 
     |████████████████████████████████| 163kB 43.0MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 7.4MB 35.2MB/s 
     |████████████████████████████████| 2.9MB 21.8MB/s 
   

In [ ]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [ ]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

03/31/2021 12:09:31 - INFO - faiss.loader -   Loading faiss with AVX2 support.
03/31/2021 12:09:31 - INFO - faiss.loader -   Loading faiss.
03/31/2021 12:09:33 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
03/31/2021 12:09:35 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.096s]
03/31/2021 12:09:35 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.435s]
03/31/2021 12:09:35 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.257s]


In [ ]:
from haystack import Finder
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

In [ ]:
# Let's first get some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
doc_dir = "data/article_txt_got"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# convert files to dicts containing documents that can be indexed to our datastore
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# We now have a list of dictionaries that we can write to our document store.
# If your texts come from a different source (e.g. a DB), you can of course skip convert_files_to_dicts() and create the dictionaries yourself.
# The default format here is: {"name": "<some-document-name>, "text": "<the-actual-text>"}

# Let's have a look at the first 3 entries:
print(dicts[:3])
# Now, let's write the docs to our DB.
document_store.write_documents(dicts)

from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [ ]:
from haystack.retriever.dense import DensePassageRetriever
dense_retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base"
)

03/31/2021 12:20:59 - INFO - filelock -   Lock 140516965944912 acquired on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


03/31/2021 12:21:00 - INFO - filelock -   Lock 140516965944912 released on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


03/31/2021 12:21:01 - INFO - filelock -   Lock 140516983104848 acquired on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


03/31/2021 12:21:02 - INFO - filelock -   Lock 140516983104848 released on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


03/31/2021 12:21:03 - INFO - filelock -   Lock 140517025288784 acquired on /root/.cache/huggingface/transformers/52774638a790c9ebc5ce11005b260f79cd4cc389abdab9eaa31e8f09d15b4f46.13b559f49587470ab6d85a7dde13174670a0b61c1b942d1489c96023f5d03772.lock


...

03/31/2021 12:21:19 - INFO - filelock -   Lock 140516965945040 released on /root/.cache/huggingface/transformers/41dac75f5df9070331cb0e4bf318c9fdeaef38d9ffd8ca80993c7db830d0c674.446ee898f4788c3ee90f8e7ee5a50281905f509e698f76dc0b583eb74ef973bd.lock


03/31/2021 12:21:27 - INFO - filelock -   Lock 140517025288784 acquired on /root/.cache/huggingface/transformers/f31ea67434695abc6c4fbe109214416d8b48a44f2fe5a0617e7faa3d6a4f8d05.be8dbf4cc0650b9c5997b3b3bc47d0d6c20749c3871e9285d3b624cd75dd9ee6.lock


03/31/2021 12:21:27 - INFO - filelock -   Lock 140517025288784 released on /root/.cache/huggingface/transformers/f31ea67434695abc6c4fbe109214416d8b48a44f2fe5a0617e7faa3d6a4f8d05.be8dbf4cc0650b9c5997b3b3bc47d0d6c20749c3871e9285d3b624cd75dd9ee6.lock


03/31/2021 12:21:28 - INFO - filelock -   Lock 140516965277904 acquired on /root/.cache/huggingface/transformers/2623d56adfe8cc7bf9275b0c620a0e271ee4004c335173bde56310dc8ea99d4f.714228ba33c6248205269978fd6d0ca0ef96508cbd4a11d894882e71d45fad7c.lock


03/31/2021 12:21:42 - INFO - filelock -   Lock 140516965277904 released on /root/.cache/huggingface/transformers/2623d56adfe8cc7bf9275b0c620a0e271ee4004c335173bde56310dc8ea99d4f.714228ba33c6248205269978fd6d0ca0ef96508cbd4a11d894882e71d45fad7c.lock


In [ ]:
reader = FARMReader(model_name_or_path="distilbert-base-uncased-distilled-squad", use_gpu=True)

03/31/2021 12:09:46 - INFO - farm.utils -   Using device: CUDA 
03/31/2021 12:09:46 - INFO - farm.utils -   Number of GPUs: 1
03/31/2021 12:09:46 - INFO - farm.utils -   Distributed Training: False
03/31/2021 12:09:46 - INFO - farm.utils -   Automatic Mixed Precision: None
03/31/2021 12:09:46 - INFO - filelock -   Lock 140517040665488 acquired on /root/.cache/huggingface/transformers/ab70e5f489e00bb2df55e4bae145e9b1c7dc794cfa0fd8228e1299d400613429.f3874c2af5400915dc843c97f502c5d30edc728e5ec3b60c4bd6958e87970f75.lock


03/31/2021 12:09:47 - INFO - filelock -   Lock 140517040665488 released on /root/.cache/huggingface/transformers/ab70e5f489e00bb2df55e4bae145e9b1c7dc794cfa0fd8228e1299d400613429.f3874c2af5400915dc843c97f502c5d30edc728e5ec3b60c4bd6958e87970f75.lock


03/31/2021 12:09:48 - INFO - filelock -   Lock 140517041029904 acquired on /root/.cache/huggingface/transformers/b00ff18397f70f871bd8f11949a3c5ffd5fb18fd6d4e3df947dc386950b8d59d.69a963759b72d26fb77afa9b7d43c9107b99dfe7ca78af52e0237c8d001c7dcf.lock


03/31/2021 12:09:56 - INFO - filelock -   Lock 140517041029904 released on /root/.cache/huggingface/transformers/b00ff18397f70f871bd8f11949a3c5ffd5fb18fd6d4e3df947dc386950b8d59d.69a963759b72d26fb77afa9b7d43c9107b99dfe7ca78af52e0237c8d001c7dcf.lock


03/31/2021 12:10:05 - INFO - filelock -   Lock 140517025873744 acquired on /root/.cache/huggingface/transformers/e12f02d630da91a0982ce6db1ad595231d155a2b725ab106971898276d842ecc.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


03/31/2021 12:10:06 - INFO - filelock -   Lock 140517025873744 released on /root/.cache/huggingface/transformers/e12f02d630da91a0982ce6db1ad595231d155a2b725ab106971898276d842ecc.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


03/31/2021 12:10:07 - INFO - filelock -   Lock 140517052884048 acquired on /root/.cache/huggingface/transformers/475d46024228961ca8770cead39e1079f135fd2441d14cf216727ffac8d41d78.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


03/31/2021 12:10:08 - INFO - filelock -   Lock 140517052884048 released on /root/.cache/huggingface/transformers/475d46024228961ca8770cead39e1079f135fd2441d14cf216727ffac8d41d78.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
03/31/2021 12:10:08 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
03/31/2021 12:10:08 - INFO - farm.utils -   Using device: CUDA 
03/31/2021 12:10:08 - INFO - farm.utils -   Number of GPUs: 1
03/31/2021 12:10:08 - INFO - farm.utils -   Distributed Training: False
03/31/2021 12:10:08 - INFO - farm.utils -   Automatic Mixed Precision: None


03/31/2021 12:10:09 - INFO - farm.infer -   Got ya 2 parallel workers to do inference ...
03/31/2021 12:10:09 - INFO - farm.infer -    0    0 
03/31/2021 12:10:09 - INFO - farm.infer -   /w\  /w\
03/31/2021 12:10:09 - INFO - farm.infer -   /'\  / \
03/31/2021 12:10:09 - INFO - farm.infer -     


In [ ]:
from haystack.pipeline import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
pipe.draw()

In [ ]:
!apt install libgraphviz-dev graphviz
!pip install pygraphviz

In [ ]:
from haystack.pipeline import Pipeline,JoinDocuments
p = Pipeline()
p.add_node(component=retriever, name="ESRetriever", inputs=["Query"])
p.add_node(component=dense_retriever, name="DPRRetriever", inputs=["Query"])
p.add_node(component=JoinDocuments(join_mode="concatenate"), name="JoinResults", inputs=["ESRetriever", "DPRRetriever"])
p.add_node(component=reader, name="QAReader", inputs=["JoinResults"])

In [ ]:
p.draw()

In [ ]:
prediction = pipe.run(query="Who is the father of Joffrey?", top_k_retriever=1,)


03/31/2021 12:28:59 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.029s]
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 25.56 Batches/s]


In [ ]:
print_answers(prediction, details="minimal")

[   {   'answer': 'Eddard',
        'context': 's Nymeria after a legendary warrior queen. She travels '
                   "with her father, Eddard, to King's Landing when he is made "
                   'Hand of the King. Before she leaves,'},
    {   'answer': 'King Robert Baratheon',
        'context': 'nerys and her dragons, but Tywin claims there is no '
                   'threat.\n'
                   'On Blackwater Bay, Melisandre reveals to Gendry that his '
                   'father was King Robert Baratheon.'},
    {   'answer': 'Tywin',
        'context': 'ghted Ser Bronn returns to the service of Tyrion '
                   'Lannister, whose father Tywin, new Hand of the King, '
                   "agrees to recognize Tyrion's accomplishments dur"},
    {   'answer': 'Ned',
        'context': '\n'
                   '====Season 1====\n'
                   'Arya accompanies her father Ned and her sister Sansa to '
                   "King's Landing. Before their departure